# Training

In [8]:
#import necessary libraries
import sys
sys.path.append('../../')

import os
import numpy as np
import torch
from PIL import Image

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import albumentations as A
from agml.train.datatools import COCOObjectDetectionDataset, get_box_transform

from agml.train.engine import train_one_epoch, evaluate, validate
import agml.train.utils
import torchvision.transforms as tra
from torchvision.transforms import functional as F
import matplotlib.pyplot as plt
import cv2
#import generate_masks as gm
import random

## Generate the data

In [9]:
from agml.data.syntheticdata import HeliosDataGenerator

In [10]:
# !mkdir ../agml/Helios
# !git clone https://github.com/PlantSimulationLab/Helios.git ../agml/Helios

In [11]:
hdg = HeliosDataGenerator()

In [12]:
hdg.canopy_param_ranges['VSPGrapevine']['grape_subdivisions'] = [[4.0, 4.0]]

In [13]:
hdg.generate_data(n_imgs=1, canopy_type='VSPGrapevine', simulation_type='rgb')

FileNotFoundError: [Errno 2] No such file or directory: '../../agml/data/../_helios/xmloutput_for_helios/tmp_canopy_params_image.xml/w'

## Load up the model

In [2]:

#define the number of class
num_classes=2 #background and grape
# load an instance segmentation model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


## Create the Dataset Object and dataloader

In [3]:
import torchvision.datasets as dset

# the dataset object
img_path="./images"
json_path="./trainval_detection.json"
import albumentations as A

from datatools import COCOObjectDetectionDataset, get_box_transform


#you can add your own albumentations transforms here 

transform_list = A.Compose([
        #A.Resize(200, 300),
        #A.CenterCrop(100, 100),
        #A.RandomCrop(80, 80),
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=(-6, 6), p=0.6),
        #A.ShiftScaleRotate(rotate_limit=50, p=0.6)
        #A.SmallestMaxSize(max_size=1292, p=1)
        #A.augmentations.transforms.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.15, rotate_limit=5, p=0.5)
        A.RandomScale(scale_limit=0.2, p=0.5)
        #A.core.composition.OneOf([A.Resize(2298, 1292, interpolation=1, always_apply=False, p=1), A.Resize(3109, 1748, interpolation=1, always_apply=False, p=1), A.Resize(2568, 1444, interpolation=1, always_apply=False, p=1), A.Resize(2839, 1596, interpolation=1, always_apply=False, p=1)],p=0.5)
        #A.augmentations.transforms.Resize(2298, 1292, interpolation=1, always_apply=False, p=1)
        #A.VerticalFlip(p=0.5),
        #A.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))


trainval = COCOObjectDetectionDataset(root = img_path,json_dir = json_path, transforms=transform_list, train=True)


# trainval = dset.CocoDetection(root = img_path,annFile = json_path)#dataset_test = GrapeBunchDataset('test', get_transform(train=False))

train, val = torch.utils.data.random_split(trainval, [2, 1]) # put 2 images in train and 1 in val for this dummy example



# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(train, batch_size=1, shuffle=True, num_workers=0, collate_fn=utils.collate_fn)

#data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=False, num_workers=4, collate_fn=utils.collate_fn)

data_loader_validate = torch.utils.data.DataLoader(train, batch_size=1, shuffle=False, num_workers=0, collate_fn=utils.collate_fn)
    

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


## Now lets train for 50 epochs

In [10]:
# now start the training process
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.009, momentum=0.9, weight_decay=0.0006)

# and a learning rate scheduler (optional)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=1)  # optional lr scheudler (gamma is 1 by default which means no scheduling)

# let's train it for 50 epochs and save after best epoch
num_epochs = 50

best_val_loss=999
for epoch in range(num_epochs):
    model.train()
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=2)
    # update the learning rate (optional)
    lr_scheduler.step()
    # evaluate and checkpoint using the validate dataset
    best_val_loss=validate(model, data_loader_validate, device=device, best_val_loss=best_val_loss)


print('Trained for '+str(num_epochs) + ' epochs')

/home/pvraja/miniconda3/envs/greenhouse/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0]  [0/2]  eta: 0:00:00  lr: 0.009000  loss: 3.7823 (3.7823)  loss_classifier: 0.7245 (0.7245)  loss_box_reg: 0.7838 (0.7838)  loss_objectness: 2.0628 (2.0628)  loss_rpn_box_reg: 0.2112 (0.2112)  time: 0.1882  data: 0.0308  max mem: 1122
Epoch: [0]  [1/2]  eta: 0:00:00  lr: 0.009000  loss: 3.7823 (3.8676)  loss_classifier: 0.7245 (0.7275)  loss_box_reg: 0.4551 (0.6194)  loss_objectness: 2.0628 (2.3046)  loss_rpn_box_reg: 0.2112 (0.2161)  time: 0.1762  data: 0.0271  max mem: 1393
Epoch: [0] Total time: 0:00:00 (0.1784 s / it)
Best Model Saved! loss: tensor(2.6444, device='cuda:0')
Epoch: [1]  [0/2]  eta: 0:00:00  lr: 0.009000  loss: 1.6583 (1.6583)  loss_classifier: 0.3448 (0.3448)  loss_box_reg: 0.3212 (0.3212)  loss_objectness: 0.5684 (0.5684)  loss_rpn_box_reg: 0.4239 (0.4239)  time: 0.1453  data: 0.0258  max mem: 1393
Epoch: [1]  [1/2]  eta: 0:00:00  lr: 0.009000  loss: 1.6583 (2.3316)  loss_classifier: 0.3448 (0.7372)  loss_box_reg: 0.3212 (0.6579)  loss_objectness: 0.5684 